In [1]:
import requests
import json
import time
import pickle
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
# import censusdata
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import math
from bs4 import BeautifulSoup as BS
import ast
from itertools import combinations
from collections import Counter
#from scipy.cluster.hierarchy 
from pyproj import Geod
from shapely.geometry import Point, LineString

%matplotlib inline

In [2]:
hidden = pd.read_pickle('../data/API_key.pkl')
api_key = hidden.loc[0][1]

In [3]:
davidson_pred_black = gpd.read_file('../data/davidson_race_by_tract/davidson_tracts_pred_black.geojson')
davidson_pred_black.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geoid       25 non-null     object  
 1   name        25 non-null     object  
 2   total_pop   25 non-null     float64 
 3   total_pop_  25 non-null     float64 
 4   white_alon  25 non-null     float64 
 5   white_al_1  25 non-null     float64 
 6   black_afri  25 non-null     float64 
 7   black_af_1  25 non-null     float64 
 8   nativeamer  25 non-null     float64 
 9   nativeam_1  25 non-null     float64 
 10  asian_alon  25 non-null     float64 
 11  asian_al_1  25 non-null     float64 
 12  hawaiian_p  25 non-null     float64 
 13  hawaiian_1  25 non-null     float64 
 14  other_alon  25 non-null     float64 
 15  other_al_1  25 non-null     float64 
 16  two_or_mor  25 non-null     float64 
 17  two_or_m_1  25 non-null     float64 
 18  two_includ  25 non-null     float64 
 19  tw

In [4]:
all_types = ['gas_station',
             'supermarket',
             'hospital',
             'clothing_store',
             'restaurant',
             'bar',] 

In [5]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  res.json()
    places.extend(results['results'])
    time.sleep(sleep_sec)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = res.json()
        places.extend(results['results'])
        time.sleep(sleep_sec)
    return places

## Because the Google API only lets you search for 1 type at a time, I've used a for loop to iterate through each type I need and create a list of those lists (nested lists, yay!)

In [6]:
location_10105 = '36.24885, -86.85387'
radius_10105 = '4534'

In [7]:
ct10105 = []
ct10105_list = []
latitude10105 = []
longitude10105 = []
name10105 = []
type10105 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10105_res = search_places_by_coordinate(location_10105, radius_10105, i, api_key) # running the API function through each type
    ct10105.append(ct10105_res) # creating a lists of the lists of each type of business
    for list in ct10105:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10105.append(element['geometry']['location']['lat'])
            longitude10105.append(element['geometry']['location']['lng'])
            name10105.append(element['name'])
            type10105.append(element['types'][0])

In [8]:
len(ct10105) # this should be 7 because we have 7 business types

6

In [9]:
len(ct10105[4]) # This would be the number of gas stations since that is the first type we iterated through

11

In [10]:
print(ct10105) #just testing to see what it looks like

[[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.2520693, 'lng': -86.8297597}, 'viewport': {'northeast': {'lat': 36.2534926302915, 'lng': -86.8284251697085}, 'southwest': {'lat': 36.2507946697085, 'lng': -86.8311231302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gas_pinlet', 'name': 'Shell', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117265797567492229445">Rooster Davis</a>'], 'photo_reference': 'Aap_uECu_NuqtF5I6FPKmLJ9lewxDBXLCSCqkea2yyaGCTxTCjSslHf1Vva0lO8ywpoN57N1wOvCM0ZfFAWGW7Zx7t2aq-uYWXQoTmgE1ZvDa6h-ZdIGcYROrMHsVjdNQCSMRhrdeBi0dMSRn_v2EsRapRlTiGlVCgQlmGa5TTrytfyUa0fd', 'width': 3096}], 'place_id': 'ChIJBQHTXnBeZIgR4BxNPRfisdI', 'plus_code': {'compound_code': '752C+R3 Nashville, TN, USA', 'global_code': '868M752C+R3'}, 'price_level': 2, 'rat

In [11]:
type10105

['gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'lodging',
 'supermarket',
 'point_of_interest',
 'clothing_store',
 'point_of_interest',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'lodging',
 'supermarket',
 'point_of_interest',
 'clothing_store',
 'point_of_interest',
 'meal_takeaway',
 'gas_st

In [12]:
df10105 = pd.DataFrame(zip(name10105, type10105, latitude10105, longitude10105),
               columns =['name', 'type', 'latitude', 'longitude'])
df10105

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Cenex KCK's,gas_station,36.279847,-86.873683
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743
4,Kwik Sak,gas_station,36.218140,-86.836695
...,...,...,...,...
85,"Joelton Hardware, Feed and Farmacy",hardware_store,36.281279,-86.875705
86,Granddaddy’s Famous Hot Chicken,restaurant,36.281556,-86.875948
87,Big Mama's Soul Food,restaurant,36.210240,-86.837661
88,Lillian and Frank's Restaurant,restaurant,36.209661,-86.839883


In [13]:
df10105.drop_duplicates(inplace=True, ignore_index=True)
df10105

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Cenex KCK's,gas_station,36.279847,-86.873683
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743
4,Kwik Sak,gas_station,36.218140,-86.836695
5,Citgo,gas_station,36.279806,-86.873688
6,Friendly Corner Market,convenience_store,36.279847,-86.873683
7,Family Dollar,supermarket,36.211460,-86.839407
8,Tony's Foodland,grocery_or_supermarket,36.280344,-86.876424
9,Fontanel,lodging,36.256853,-86.829741


In [14]:
df10105['tract'] = 'census_tract_10105'
df10105

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10105
1,Cenex KCK's,gas_station,36.279847,-86.873683,census_tract_10105
2,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_10105
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743,census_tract_10105
4,Kwik Sak,gas_station,36.218140,-86.836695,census_tract_10105
5,Citgo,gas_station,36.279806,-86.873688,census_tract_10105
6,Friendly Corner Market,convenience_store,36.279847,-86.873683,census_tract_10105
7,Family Dollar,supermarket,36.211460,-86.839407,census_tract_10105
8,Tony's Foodland,grocery_or_supermarket,36.280344,-86.876424,census_tract_10105
9,Fontanel,lodging,36.256853,-86.829741,census_tract_10105


In [15]:
test_map = folium.Map([36.24885, -86.85388])
for row_index, row_values in df10105.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    tool = [str(row_values['name']), str(row_values['type'])]
    
    marker = folium.Marker(
        location = loc, 
        tooltip = tool) 

    marker.add_to(test_map)
test_map

# Now that I've created the DataFrame for the first census tract, I will do the same for the others

In [16]:
location_10106 = '36.25371, -86.80361'
radius_10106 = '4000'

In [17]:
ct10106 = []
ct10106_list = []
latitude10106 = []
longitude10106 = []
name10106 = []
type10106 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10106_res = search_places_by_coordinate(location_10106, radius_10106, i, api_key) # running the API function through each type
    ct10106.append(ct10106_res) # creating a lists of the lists of each type of business
    for list in ct10106:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10106.append(element['geometry']['location']['lat'])
            longitude10106.append(element['geometry']['location']['lng'])
            name10106.append(element['name'])
            type10106.append(element['types'][0])

In [18]:
name10106

['Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Whites Creek Wellness & Rehabilitation Center',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Whites Creek Wellness & Rehabilitation Center',
 'Fontanel',
 'Walmart Supercenter',
 'Xclusive Make-Up/SkinCare Studio',
 'Rainbow Shops',
 'Cintas Uniform Services',
 'Vintage Creek Boutique',
 'G-mommas Big Sale',
 'The Plush Vault',
 'Family Dollar',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Whites Creek Wellness & Rehabilitation Center',
 'Fontanel',
 'W

In [19]:
df10106 = pd.DataFrame((zip(name10106, type10106, latitude10106, longitude10106)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [20]:
df10106.drop_duplicates(inplace=True, ignore_index=True)
df10106

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Tri-Star Energy,convenience_store,36.276629,-86.795924
2,Shell,gas_station,36.277810,-86.796850
3,Citgo,gas_station,36.232554,-86.781664
4,TERRY'S,convenience_store,36.232562,-86.781676
5,Marathon,gas_station,36.276661,-86.795952
6,Little Barn Market,convenience_store,36.231354,-86.781349
7,A a Mart,convenience_store,36.218555,-86.804160
8,Family Dollar,supermarket,36.219835,-86.805205
9,Whites Creek Wellness & Rehabilitation Center,hospital,36.238445,-86.802270


In [21]:
df10106['tract'] = 'census_tract_10106'
df10106

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10106
1,Tri-Star Energy,convenience_store,36.276629,-86.795924,census_tract_10106
2,Shell,gas_station,36.277810,-86.796850,census_tract_10106
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10106
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10106
5,Marathon,gas_station,36.276661,-86.795952,census_tract_10106
6,Little Barn Market,convenience_store,36.231354,-86.781349,census_tract_10106
7,A a Mart,convenience_store,36.218555,-86.804160,census_tract_10106
8,Family Dollar,supermarket,36.219835,-86.805205,census_tract_10106
9,Whites Creek Wellness & Rehabilitation Center,hospital,36.238445,-86.802270,census_tract_10106


In [22]:
location_10903 = '36.24932, -86.76382'
radius_10903 = '2716'

In [23]:
ct10903 = []
ct10903_list = []
latitude10903 = []
longitude10903 = []
name10903 = []
type10903 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10903_res = search_places_by_coordinate(location_10903, radius_10903, i, api_key) # running the API function through each type
    ct10903.append(ct10903_res) # creating a lists of the lists of each type of business
    for list in ct10903:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10903.append(element['geometry']['location']['lat'])
            longitude10903.append(element['geometry']['location']['lng'])
            name10903.append(element['name'])
            type10903.append(element['types'][0])

In [24]:
name10903

['Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'H G Hill Food Stores',
 'Family Dollar',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'SunCrest Home Health',
 'DaVita Briley Parkway Dialysis',
 'Capstone Pediatrics Skyline',
 'Skyline medical centrr',
 'Skyline Medical Center',
 'Skyline medical Plaza',
 'Nashville Neurosurgery Associates',
 'TRISTACA HYDE SLP',
 'Hospital tgt',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 

In [25]:
df10903 = pd.DataFrame(zip(name10903, type10903, latitude10903, longitude10903),
               columns =['name', 'type', 'latitude', 'longitude'])

In [26]:
df10903.drop_duplicates(inplace=True, ignore_index=True)
df10903

,name,type,latitude,longitude
0,Exxon,gas_station,36.262900,-86.757836
1,MAPCO,gas_station,36.229043,-86.758997
2,Shell,gas_station,36.268857,-86.755242
3,Citgo,gas_station,36.232554,-86.781664
4,Shell,gas_station,36.242914,-86.736170
...,...,...,...,...
62,Captain D's,restaurant,36.227010,-86.759350
63,SideKicks Café,cafe,36.248142,-86.735112
64,PoeBoy Luncheon,restaurant,36.226104,-86.760394
65,Ambrosia Hookah Bar and Lounge,bar,36.230037,-86.761003


In [27]:
df10903['tract'] = 'census_tract_10903'
df10903

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.262900,-86.757836,census_tract_10903
1,MAPCO,gas_station,36.229043,-86.758997,census_tract_10903
2,Shell,gas_station,36.268857,-86.755242,census_tract_10903
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10903
4,Shell,gas_station,36.242914,-86.736170,census_tract_10903
...,...,...,...,...,...
62,Captain D's,restaurant,36.227010,-86.759350,census_tract_10903
63,SideKicks Café,cafe,36.248142,-86.735112,census_tract_10903
64,PoeBoy Luncheon,restaurant,36.226104,-86.760394,census_tract_10903
65,Ambrosia Hookah Bar and Lounge,bar,36.230037,-86.761003,census_tract_10903


In [28]:
location_10904 = '36.23431, -86.77321'
radius_10904 = '2203'

In [29]:
ct10904 = []
ct10904_list = []
latitude10904 = []
longitude10904 = []
name10904 = []
type10904 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10904_res = search_places_by_coordinate(location_10904, radius_10904, i, api_key) # running the API function through each type
    ct10904.append(ct10904_res) # creating a lists of the lists of each type of business
    for list in ct10904:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10904.append(element['geometry']['location']['lat'])
            longitude10904.append(element['geometry']['location']['lng'])
            name10904.append(element['name'])
            type10904.append(element['types'][0])

In [30]:
name10904

['MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'Skyline medical centrr',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'Skyline medical centrr',
 'Walmart Supercenter',
 'Walgreens',
 'Xclusive Make-Up/SkinCare Studio',
 'Rainbo

In [31]:
df10904 = pd.DataFrame(zip(name10904, type10904, latitude10904, longitude10904),
               columns =['name', 'type', 'latitude', 'longitude'])

In [32]:
df10904.drop_duplicates(inplace=True, ignore_index=True)
df10904

,name,type,latitude,longitude
0,MAPCO,gas_station,36.229043,-86.758997
1,Citgo,gas_station,36.232554,-86.781664
2,MAPCO Mart,gas_station,36.218611,-86.762394
3,Murphy Express,gas_station,36.243486,-86.757311
4,TERRY'S,convenience_store,36.232562,-86.781676
5,Little Barn Market,convenience_store,36.231354,-86.781349
6,Exxon,gas_station,36.232835,-86.759098
7,Dickerson Road Citgo,gas_station,36.229185,-86.759641
8,Saint George Market,gas_station,36.231935,-86.758394
9,G dragon mart,gas_station,36.216211,-86.763359


In [33]:
df10904['tract'] = 'census_tract_10904'
df10904

,name,type,latitude,longitude,tract
0,MAPCO,gas_station,36.229043,-86.758997,census_tract_10904
1,Citgo,gas_station,36.232554,-86.781664,census_tract_10904
2,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_10904
3,Murphy Express,gas_station,36.243486,-86.757311,census_tract_10904
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10904
5,Little Barn Market,convenience_store,36.231354,-86.781349,census_tract_10904
6,Exxon,gas_station,36.232835,-86.759098,census_tract_10904
7,Dickerson Road Citgo,gas_station,36.229185,-86.759641,census_tract_10904
8,Saint George Market,gas_station,36.231935,-86.758394,census_tract_10904
9,G dragon mart,gas_station,36.216211,-86.763359,census_tract_10904


In [34]:
location_11001 = '36.22296, -86.75982'
radius_11001 = '2701'

In [35]:
ct11001 = []
ct11001_list = []
latitude11001 = []
longitude11001 = []
name11001 = []
type11001 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct11001_res = search_places_by_coordinate(location_11001, radius_11001, i, api_key) # running the API function through each type
    ct11001.append(ct11001_res) # creating a lists of the lists of each type of business
    for list in ct11001:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude11001.append(element['geometry']['location']['lat'])
            longitude11001.append(element['geometry']['location']['lng'])
            name11001.append(element['name'])
            type11001.append(element['types'][0])

In [36]:
df11001 = pd.DataFrame(zip(name11001, type11001, latitude11001, longitude11001),
               columns =['name', 'type', 'latitude', 'longitude'])

In [37]:
df11001.drop_duplicates(inplace=True, ignore_index=True)
df11001

,name,type,latitude,longitude
0,MAPCO Mart,gas_station,36.218611,-86.762394
1,MAPCO,gas_station,36.229043,-86.758997
2,Citgo,gas_station,36.232554,-86.781664
3,Shell,gas_station,36.208675,-86.778131
4,Murphy Express,gas_station,36.243486,-86.757311
...,...,...,...,...
84,La Parrilla Fresh Mexican Bar & Grill,meal_takeaway,36.207325,-86.779411
85,Taco Bell,meal_takeaway,36.205150,-86.778461
86,TRVTH,bar,36.222849,-86.760892
87,The Beer Joint,bar,36.226154,-86.760673


In [38]:
df11001['tract'] = 'census_tract_11001'
df11001

,name,type,latitude,longitude,tract
0,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_11001
1,MAPCO,gas_station,36.229043,-86.758997,census_tract_11001
2,Citgo,gas_station,36.232554,-86.781664,census_tract_11001
3,Shell,gas_station,36.208675,-86.778131,census_tract_11001
4,Murphy Express,gas_station,36.243486,-86.757311,census_tract_11001
...,...,...,...,...,...
84,La Parrilla Fresh Mexican Bar & Grill,meal_takeaway,36.207325,-86.779411,census_tract_11001
85,Taco Bell,meal_takeaway,36.205150,-86.778461,census_tract_11001
86,TRVTH,bar,36.222849,-86.760892,census_tract_11001
87,The Beer Joint,bar,36.226154,-86.760673,census_tract_11001


In [39]:
location_114 = '36.19852, -86.73378'
radius_114 = '1998'

In [40]:
ct114 = []
ct114_list = []
latitude114 = []
longitude114 = []
name114 = []
type114 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct114_res = search_places_by_coordinate(location_114, radius_114, i, api_key) # running the API function through each type
    ct114.append(ct114_res) # creating a lists of the lists of each type of business
    for list in ct114:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude114.append(element['geometry']['location']['lat'])
            longitude114.append(element['geometry']['location']['lng'])
            name114.append(element['name'])
            type114.append(element['types'][0])

In [41]:
df114 = pd.DataFrame(zip(name114, type114, latitude114, longitude114),
               columns =['name', 'type', 'latitude', 'longitude'])

In [42]:
df114.drop_duplicates(inplace=True, ignore_index=True)
df114

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.210014,-86.733138
1,Kwik Sak,convenience_store,36.214093,-86.730432
2,Kroger Fuel Center,gas_station,36.185340,-86.747265
3,Speedway,gas_station,36.196900,-86.741704
4,Citgo,gas_station,36.205030,-86.724112
...,...,...,...,...
108,Fran's East Side,night_club,36.188661,-86.728656
109,Pearl Diver,bar,36.189090,-86.746377
110,The Authentique,bar,36.187717,-86.746219
111,TailGate Brewery East,meal_takeaway,36.186339,-86.746997


In [43]:
df114['tract'] = 'census_tract_114'
df114

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.210014,-86.733138,census_tract_114
1,Kwik Sak,convenience_store,36.214093,-86.730432,census_tract_114
2,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_114
3,Speedway,gas_station,36.196900,-86.741704,census_tract_114
4,Citgo,gas_station,36.205030,-86.724112,census_tract_114
...,...,...,...,...,...
108,Fran's East Side,night_club,36.188661,-86.728656,census_tract_114
109,Pearl Diver,bar,36.189090,-86.746377,census_tract_114
110,The Authentique,bar,36.187717,-86.746219,census_tract_114
111,TailGate Brewery East,meal_takeaway,36.186339,-86.746997,census_tract_114


In [44]:
location_118 = '36.18601, -86.76437'
radius_118 = '1818'

In [45]:
ct118 = []
ct118_list = []
latitude118 = []
longitude118= []
name118 = []
type118 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct118_res = search_places_by_coordinate(location_118, radius_118, i, api_key) # running the API function through each type
    ct118.append(ct118_res) # creating a lists of the lists of each type of business
    for list in ct118:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude118.append(element['geometry']['location']['lat'])
            longitude118.append(element['geometry']['location']['lng'])
            name118.append(element['name'])
            type118.append(element['types'][0])

In [46]:
df118 = pd.DataFrame(zip(name118, type118, latitude118, longitude118),
               columns =['name', 'type', 'latitude', 'longitude'])

In [47]:
df118.drop_duplicates(inplace=True, ignore_index=True)
df118

,name,type,latitude,longitude
0,Trillium,gas_station,36.178444,-86.774921
1,Kroger Fuel Center,gas_station,36.185340,-86.747265
2,Express Food Mart,convenience_store,36.177783,-86.749592
3,Marathon,gas_station,36.177779,-86.749787
4,TA Travel Center,gas_station,36.171075,-86.771986
...,...,...,...,...
140,The Authentique,bar,36.187717,-86.746219
141,Pearl Diver,bar,36.189090,-86.746377
142,Van Gogh's Wine Bar,bar,36.177012,-86.748748
143,Monday Night Preservation Co.,bar,36.179586,-86.782982


In [48]:
df118['tract'] = 'census_tract_118'
df118

,name,type,latitude,longitude,tract
0,Trillium,gas_station,36.178444,-86.774921,census_tract_118
1,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_118
2,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_118
3,Marathon,gas_station,36.177779,-86.749787,census_tract_118
4,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_118
...,...,...,...,...,...
140,The Authentique,bar,36.187717,-86.746219,census_tract_118
141,Pearl Diver,bar,36.189090,-86.746377,census_tract_118
142,Van Gogh's Wine Bar,bar,36.177012,-86.748748,census_tract_118
143,Monday Night Preservation Co.,bar,36.179586,-86.782982,census_tract_118


In [49]:
location_119= '36.17908, -86.75906'
radius_119= '1658'

In [50]:
ct119 = []
ct119_list = []
latitude119 = []
longitude119= []
name119 = []
type119 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct119_res = search_places_by_coordinate(location_119, radius_119, i, api_key) # running the API function through each type
    ct119.append(ct119_res) # creating a lists of the lists of each type of business
    for list in ct119:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude119.append(element['geometry']['location']['lat'])
            longitude119.append(element['geometry']['location']['lng'])
            name119.append(element['name'])
            type119.append(element['types'][0])

In [51]:
df119 = pd.DataFrame(zip(name119, type119, latitude119, longitude119),
               columns =['name', 'type', 'latitude', 'longitude'])

In [52]:
df119.drop_duplicates(inplace=True, ignore_index=True)
df119

,name,type,latitude,longitude
0,TA Travel Center,gas_station,36.171075,-86.771986
1,Express Food Mart,convenience_store,36.177783,-86.749592
2,Marathon,gas_station,36.177779,-86.749787
3,Shell,gas_station,36.171552,-86.751873
4,Kroger Fuel Center,gas_station,36.185340,-86.747265
...,...,...,...,...
139,The Authentique,bar,36.187717,-86.746219
140,Urban Cowboy Public House,night_club,36.177307,-86.741657
141,ROBERTASPIZZA,bar,36.177328,-86.741592
142,Pearl Diver,bar,36.189090,-86.746377


In [53]:
df119['tract'] = 'census_tract_119'
df119

,name,type,latitude,longitude,tract
0,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_119
1,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_119
2,Marathon,gas_station,36.177779,-86.749787,census_tract_119
3,Shell,gas_station,36.171552,-86.751873,census_tract_119
4,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_119
...,...,...,...,...,...
139,The Authentique,bar,36.187717,-86.746219,census_tract_119
140,Urban Cowboy Public House,night_club,36.177307,-86.741657,census_tract_119
141,ROBERTASPIZZA,bar,36.177328,-86.741592,census_tract_119
142,Pearl Diver,bar,36.189090,-86.746377,census_tract_119


In [54]:
location_126= '36.18710, -86.77198'
radius_126= '1672'

In [55]:
ct126 = []
ct126_list = []
latitude126 = []
longitude126= []
name126 = []
type126 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct126_res = search_places_by_coordinate(location_126, radius_126, i, api_key) # running the API function through each type
    ct126.append(ct126_res) # creating a lists of the lists of each type of business
    for list in ct126:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude126.append(element['geometry']['location']['lat'])
            longitude126.append(element['geometry']['location']['lng'])
            name126.append(element['name'])
            type126.append(element['types'][0])

In [56]:
df126 = pd.DataFrame(zip(name126, type126, latitude126, longitude126),
               columns =['name', 'type', 'latitude', 'longitude'])

In [57]:
df126.drop_duplicates(inplace=True, ignore_index=True)
df126

,name,type,latitude,longitude
0,Trillium,gas_station,36.178444,-86.774921
1,Z Mart,gas_station,36.184732,-86.773392
2,Citgo,gas_station,36.183447,-86.773557
3,Safelite-Nashville,gas_station,36.188180,-86.777185
4,MyStik,gas_station,36.176671,-86.774879
...,...,...,...,...
61,Lyra,restaurant,36.184520,-86.754234
62,Saint Stephen,restaurant,36.177922,-86.786424
63,Magnolia,bar,36.182302,-86.766555
64,Music City Rush LLC,bar,36.174824,-86.773682


In [58]:
df126['tract'] = 'census_tract_126'
df126

,name,type,latitude,longitude,tract
0,Trillium,gas_station,36.178444,-86.774921,census_tract_126
1,Z Mart,gas_station,36.184732,-86.773392,census_tract_126
2,Citgo,gas_station,36.183447,-86.773557,census_tract_126
3,Safelite-Nashville,gas_station,36.188180,-86.777185,census_tract_126
4,MyStik,gas_station,36.176671,-86.774879,census_tract_126
...,...,...,...,...,...
61,Lyra,restaurant,36.184520,-86.754234,census_tract_126
62,Saint Stephen,restaurant,36.177922,-86.786424,census_tract_126
63,Magnolia,bar,36.182302,-86.766555,census_tract_126
64,Music City Rush LLC,bar,36.174824,-86.773682,census_tract_126


In [59]:
location_12701= '36.22121, -86.79463'
radius_12701= '2946'

In [60]:
ct12701 = []
ct12701_list = []
latitude12701 = []
longitude12701 = []
name12701 = []
type12701 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12701_res = search_places_by_coordinate(location_12701, radius_12701, i, api_key) # running the API function through each type
    ct12701.append(ct12701_res) # creating a lists of the lists of each type of business
    for list in ct12701:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12701.append(element['geometry']['location']['lat'])
            longitude12701.append(element['geometry']['location']['lng'])
            name12701.append(element['name'])
            type12701.append(element['types'][0])

In [61]:
df12701 = pd.DataFrame(zip(name12701, type12701, latitude12701, longitude12701),
               columns =['name', 'type', 'latitude', 'longitude'])

In [62]:
df12701.drop_duplicates(inplace=True, ignore_index=True)
df12701

,name,type,latitude,longitude
0,Citgo,gas_station,36.232554,-86.781664
1,Shell,gas_station,36.208675,-86.778131
2,MAPCO Mart,gas_station,36.218611,-86.762394
3,TERRY'S,convenience_store,36.232562,-86.781676
4,Tiger Mart,convenience_store,36.207392,-86.778339
5,A a Mart,convenience_store,36.218555,-86.804160
6,Victory Fuels,gas_station,36.207378,-86.791496
7,Little Barn Market,convenience_store,36.231354,-86.781349
8,Exxon,gas_station,36.207427,-86.778294
9,Exxon,gas_station,36.207173,-86.778311


In [63]:
df12701['tract'] = 'census_tract_12701'
df12701

,name,type,latitude,longitude,tract
0,Citgo,gas_station,36.232554,-86.781664,census_tract_12701
1,Shell,gas_station,36.208675,-86.778131,census_tract_12701
2,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_12701
3,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_12701
4,Tiger Mart,convenience_store,36.207392,-86.778339,census_tract_12701
5,A a Mart,convenience_store,36.218555,-86.804160,census_tract_12701
6,Victory Fuels,gas_station,36.207378,-86.791496,census_tract_12701
7,Little Barn Market,convenience_store,36.231354,-86.781349,census_tract_12701
8,Exxon,gas_station,36.207427,-86.778294,census_tract_12701
9,Exxon,gas_station,36.207173,-86.778311,census_tract_12701


In [64]:
location_12702= '36.20596, -86.79612'
radius_12702= '2824'

In [65]:
ct12702 = []
ct12702_list = []
latitude12702 = []
longitude12702 = []
name12702 = []
type12702 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12702_res = search_places_by_coordinate(location_12702, radius_12702, i, api_key) # running the API function through each type
    ct12702.append(ct12702_res) # creating a lists of the lists of each type of business
    for list in ct12702:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12702.append(element['geometry']['location']['lat'])
            longitude12702.append(element['geometry']['location']['lng'])
            name12702.append(element['name'])
            type12702.append(element['types'][0])

In [66]:
df12702 = pd.DataFrame(zip(name12702, type12702, latitude12702, longitude12702),
               columns =['name', 'type', 'latitude', 'longitude'])

In [67]:
df12702.drop_duplicates(inplace=True, ignore_index=True)
df12702

,name,type,latitude,longitude
0,Exxon,gas_station,36.188416,-86.798098
1,Shell,gas_station,36.208675,-86.778131
2,Mobil,gas_station,36.188316,-86.798082
3,Tiger Mart,convenience_store,36.188453,-86.798177
4,Tiger Mart,convenience_store,36.207392,-86.778339
...,...,...,...,...
76,Nashville Pizza & Pasta,meal_delivery,36.192030,-86.772350
77,Saucey Baby 615,restaurant,36.190936,-86.772528
78,House of Hickory BBQ,restaurant,36.190624,-86.772532
79,Maxwell's Lounge,night_club,36.189363,-86.800004


In [68]:
df12702['tract'] = 'census_tract_12702'
df12702

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.188416,-86.798098,census_tract_12702
1,Shell,gas_station,36.208675,-86.778131,census_tract_12702
2,Mobil,gas_station,36.188316,-86.798082,census_tract_12702
3,Tiger Mart,convenience_store,36.188453,-86.798177,census_tract_12702
4,Tiger Mart,convenience_store,36.207392,-86.778339,census_tract_12702
...,...,...,...,...,...
76,Nashville Pizza & Pasta,meal_delivery,36.192030,-86.772350,census_tract_12702
77,Saucey Baby 615,restaurant,36.190936,-86.772528,census_tract_12702
78,House of Hickory BBQ,restaurant,36.190624,-86.772532,census_tract_12702
79,Maxwell's Lounge,night_club,36.189363,-86.800004,census_tract_12702


In [69]:
location_12801= '36.20801, -86.83523'
radius_12801= '2915'

In [70]:
ct12801 = []
ct12801_list = []
latitude12801 = []
longitude12801 = []
name12801 = []
type12801 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12801_res = search_places_by_coordinate(location_12801, radius_12801, i, api_key) # running the API function through each type
    ct12801.append(ct12801_res) # creating a lists of the lists of each type of business
    for list in ct12801:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12801.append(element['geometry']['location']['lat'])
            longitude12801.append(element['geometry']['location']['lng'])
            name12801.append(element['name'])
            type12801.append(element['types'][0])

In [71]:
df12801 = pd.DataFrame(zip(name12801, type12801, latitude12801, longitude12801),
               columns =['name', 'type', 'latitude', 'longitude'])

In [72]:
df12801.drop_duplicates(inplace=True, ignore_index=True)
df12801

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.210674,-86.838564
1,Little Barn Market,gas_station,36.209159,-86.839701
2,F & B Mart,gas_station,36.207746,-86.839515
3,Citigo,gas_station,36.208055,-86.839666
4,Bordeaux Grab&go,gas_station,36.209226,-86.839838
5,Kwik Sak,gas_station,36.218140,-86.836695
6,Zack's Express,gas_station,36.196496,-86.835309
7,Exxon,gas_station,36.194220,-86.831207
8,Kwik Sak,gas_station,36.193146,-86.831060
9,Z Mart,gas_station,36.187249,-86.819371


In [73]:
df12801['tract'] = 'census_tract_12801'
df12801

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_12801
1,Little Barn Market,gas_station,36.209159,-86.839701,census_tract_12801
2,F & B Mart,gas_station,36.207746,-86.839515,census_tract_12801
3,Citigo,gas_station,36.208055,-86.839666,census_tract_12801
4,Bordeaux Grab&go,gas_station,36.209226,-86.839838,census_tract_12801
5,Kwik Sak,gas_station,36.218140,-86.836695,census_tract_12801
6,Zack's Express,gas_station,36.196496,-86.835309,census_tract_12801
7,Exxon,gas_station,36.194220,-86.831207,census_tract_12801
8,Kwik Sak,gas_station,36.193146,-86.831060,census_tract_12801
9,Z Mart,gas_station,36.187249,-86.819371,census_tract_12801


In [74]:
location_12802= '36.19057, -86.85533'
radius_12802= '3509'

In [75]:
ct12802 = []
ct12802_list = []
latitude12802 = []
longitude12802 = []
name12802 = []
type12802 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12802_res = search_places_by_coordinate(location_12802, radius_12802, i, api_key) # running the API function through each type
    ct12802.append(ct12802_res) # creating a lists of the lists of each type of business
    for list in ct12802:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12802.append(element['geometry']['location']['lat'])
            longitude12802.append(element['geometry']['location']['lng'])
            name12802.append(element['name'])
            type12802.append(element['types'][0])

In [76]:
df12802 = pd.DataFrame(zip(name12802, type12802, latitude12802, longitude12802),
               columns =['name', 'type', 'latitude', 'longitude'])

In [77]:
df12802.drop_duplicates(inplace=True, ignore_index=True)
df12802

,name,type,latitude,longitude
0,Pilot Travel Center,gas_station,36.172260,-86.865304
1,Magellan Midstream Partners LP,gas_station,36.169289,-86.863084
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Marathon Petroleum 51st,gas_station,36.165596,-86.849273
4,Little Barn Market,gas_station,36.209159,-86.839701
...,...,...,...,...
83,Rock'n Dough Pizza & Brewery,restaurant,36.160623,-86.849365
84,The Nashville SmokeHouse,restaurant,36.205375,-86.821505
85,The Centennial,bar,36.162712,-86.850104
86,A Notable Event,night_club,36.180915,-86.822564


In [78]:
df12802['tract'] = 'census_tract_12802'
df12802

,name,type,latitude,longitude,tract
0,Pilot Travel Center,gas_station,36.172260,-86.865304,census_tract_12802
1,Magellan Midstream Partners LP,gas_station,36.169289,-86.863084,census_tract_12802
2,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_12802
3,Marathon Petroleum 51st,gas_station,36.165596,-86.849273,census_tract_12802
4,Little Barn Market,gas_station,36.209159,-86.839701,census_tract_12802
...,...,...,...,...,...
83,Rock'n Dough Pizza & Brewery,restaurant,36.160623,-86.849365,census_tract_12802
84,The Nashville SmokeHouse,restaurant,36.205375,-86.821505,census_tract_12802
85,The Centennial,bar,36.162712,-86.850104,census_tract_12802
86,A Notable Event,night_club,36.180915,-86.822564,census_tract_12802


In [79]:
location_13601= '36.16390, -86.82741'
radius_13601= '2448'

In [80]:
ct13601 = []
ct13601_list = []
latitude13601 = []
longitude13601 = []
name13601 = []
type13601 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct13601_res = search_places_by_coordinate(location_13601, radius_13601, i, api_key) # running the API function through each type
    ct13601.append(ct13601_res) # creating a lists of the lists of each type of business
    for list in ct13601:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude13601.append(element['geometry']['location']['lat'])
            longitude13601.append(element['geometry']['location']['lng'])
            name13601.append(element['name'])
            type13601.append(element['types'][0])

In [81]:
df13601 = pd.DataFrame(zip(name13601, type13601, latitude13601, longitude13601),
               columns =['name', 'type', 'latitude', 'longitude'])

In [82]:
df13601.drop_duplicates(inplace=True, ignore_index=True)
df13601

,name,type,latitude,longitude
0,Petroleum Equipment Co Inc,gas_station,36.153636,-86.835094
1,Shell,gas_station,36.152982,-86.842490
2,Marathon Petroleum 51st,gas_station,36.165596,-86.849273
3,Shell,gas_station,36.156657,-86.806903
4,MAPCO,gas_station,36.152134,-86.838028
...,...,...,...,...
171,Betty's Grill,bar,36.152504,-86.846100
172,Nations Bar and Grill,bar,36.155662,-86.849056
173,G-SPOT,night_club,36.179914,-86.813541
174,The Last Hole Sports Bar & Grill,bar,36.170684,-86.802673


In [83]:
df13601['tract'] = 'census_tract_13601'
df13601

,name,type,latitude,longitude,tract
0,Petroleum Equipment Co Inc,gas_station,36.153636,-86.835094,census_tract_13601
1,Shell,gas_station,36.152982,-86.842490,census_tract_13601
2,Marathon Petroleum 51st,gas_station,36.165596,-86.849273,census_tract_13601
3,Shell,gas_station,36.156657,-86.806903,census_tract_13601
4,MAPCO,gas_station,36.152134,-86.838028,census_tract_13601
...,...,...,...,...,...
171,Betty's Grill,bar,36.152504,-86.846100,census_tract_13601
172,Nations Bar and Grill,bar,36.155662,-86.849056,census_tract_13601
173,G-SPOT,night_club,36.179914,-86.813541,census_tract_13601
174,The Last Hole Sports Bar & Grill,bar,36.170684,-86.802673,census_tract_13601


In [84]:
location_13602= '36.17144, -86.82851'
radius_13602= '1885'

In [85]:
ct13602 = []
ct13602_list = []
latitude13602 = []
longitude13602 = []
name13602 = []
type13602 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct13602_res = search_places_by_coordinate(location_13602, radius_13602, i, api_key) # running the API function through each type
    ct13602.append(ct13602_res) # creating a lists of the lists of each type of business
    for list in ct13602:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude13602.append(element['geometry']['location']['lat'])
            longitude13602.append(element['geometry']['location']['lng'])
            name13602.append(element['name'])
            type13602.append(element['types'][0])

In [86]:
df13602 = pd.DataFrame(zip(name13602, type13602, latitude13602, longitude13602),
               columns =['name', 'type', 'latitude', 'longitude'])

In [87]:
df13602.drop_duplicates(inplace=True, ignore_index=True)
df13602

,name,type,latitude,longitude
0,Marathon Gas,gas_station,36.167786,-86.821652
1,Marathon Gas,gas_station,36.167704,-86.820956
2,Pacific Pride,gas_station,36.179149,-86.824220
3,A header Lane Church of Christ,gas_station,36.171479,-86.816625
4,Schrader. Lane church,gas_station,36.171479,-86.816625
5,Family Dollar,supermarket,36.168421,-86.817053
6,Family Dollar,supermarket,36.158168,-86.830153
7,Resha's Bi-Rite Market,grocery_or_supermarket,36.159497,-86.814018
8,Queen Washington Building,hospital,36.168180,-86.832014
9,Curahealth Nashville,hospital,36.177544,-86.846419


In [88]:
df13602['tract'] = 'census_tract_13602'
df13602

,name,type,latitude,longitude,tract
0,Marathon Gas,gas_station,36.167786,-86.821652,census_tract_13602
1,Marathon Gas,gas_station,36.167704,-86.820956,census_tract_13602
2,Pacific Pride,gas_station,36.179149,-86.824220,census_tract_13602
3,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_13602
4,Schrader. Lane church,gas_station,36.171479,-86.816625,census_tract_13602
5,Family Dollar,supermarket,36.168421,-86.817053,census_tract_13602
6,Family Dollar,supermarket,36.158168,-86.830153,census_tract_13602
7,Resha's Bi-Rite Market,grocery_or_supermarket,36.159497,-86.814018,census_tract_13602
8,Queen Washington Building,hospital,36.168180,-86.832014,census_tract_13602
9,Curahealth Nashville,hospital,36.177544,-86.846419,census_tract_13602


In [89]:
location_137= '36.19115, -86.80853'
radius_137= '2551'

In [90]:
ct137 = []
ct137_list = []
latitude137 = []
longitude137 = []
name137 = []
type137 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct137_res = search_places_by_coordinate(location_137, radius_137, i, api_key) # running the API function through each type
    ct137.append(ct137_res) # creating a lists of the lists of each type of business
    for list in ct137:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude137.append(element['geometry']['location']['lat'])
            longitude137.append(element['geometry']['location']['lng'])
            name137.append(element['name'])
            type137.append(element['types'][0])

In [91]:
df137 = pd.DataFrame(zip(name137, type137, latitude137, longitude137),
               columns =['name', 'type', 'latitude', 'longitude'])

In [92]:
df137.drop_duplicates(inplace=True, ignore_index=True)
df137

,name,type,latitude,longitude
0,Exxon,gas_station,36.188416,-86.798098
1,Mobil,gas_station,36.188316,-86.798082
2,Kroger Fuel Center,gas_station,36.175368,-86.793211
3,Tiger Mart,convenience_store,36.188453,-86.798177
4,bp,gas_station,36.172959,-86.791613
...,...,...,...,...
117,Mother's Ruin,bar,36.176075,-86.790393
118,bar,bar,36.175811,-86.790228
119,swerf333,bar,36.175811,-86.790228
120,EG&MC,bar,36.172862,-86.793766


In [93]:
df137['tract'] = 'census_tract_137'
df137

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.188416,-86.798098,census_tract_137
1,Mobil,gas_station,36.188316,-86.798082,census_tract_137
2,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_137
3,Tiger Mart,convenience_store,36.188453,-86.798177,census_tract_137
4,bp,gas_station,36.172959,-86.791613,census_tract_137
...,...,...,...,...,...
117,Mother's Ruin,bar,36.176075,-86.790393,census_tract_137
118,bar,bar,36.175811,-86.790228,census_tract_137
119,swerf333,bar,36.175811,-86.790228,census_tract_137
120,EG&MC,bar,36.172862,-86.793766,census_tract_137


In [94]:
location_138= '36.17565, -86.81480'
radius_138= '1559'

In [95]:
ct138 = []
ct138_list = []
latitude138 = []
longitude138 = []
name138 = []
type138 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct138_res = search_places_by_coordinate(location_138, radius_138, i, api_key) # running the API function through each type
    ct138.append(ct138_res) # creating a lists of the lists of each type of business
    for list in ct138:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude138.append(element['geometry']['location']['lat'])
            longitude138.append(element['geometry']['location']['lng'])
            name138.append(element['name'])
            type138.append(element['types'][0])

In [96]:
df138 = pd.DataFrame(zip(name138, type138, latitude138, longitude138),
               columns =['name', 'type', 'latitude', 'longitude'])

In [97]:
df138.drop_duplicates(inplace=True, ignore_index=True)
df138

,name,type,latitude,longitude
0,Pauls Market,gas_station,36.169698,-86.807180
1,A header Lane Church of Christ,gas_station,36.171479,-86.816625
2,Schrader. Lane church,gas_station,36.171479,-86.816625
3,Pacific Pride,gas_station,36.179149,-86.824220
4,Kings Market,gas_station,36.169579,-86.807004
...,...,...,...,...
63,Rocky's Wing Shack,restaurant,36.178329,-86.798754
64,Alexandria Grill & Pizza,restaurant,36.180131,-86.798522
65,A Notable Event,night_club,36.180915,-86.822564
66,G-SPOT,night_club,36.179914,-86.813541


In [98]:
df138['tract'] = 'census_tract_138'
df138

,name,type,latitude,longitude,tract
0,Pauls Market,gas_station,36.169698,-86.807180,census_tract_138
1,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_138
2,Schrader. Lane church,gas_station,36.171479,-86.816625,census_tract_138
3,Pacific Pride,gas_station,36.179149,-86.824220,census_tract_138
4,Kings Market,gas_station,36.169579,-86.807004,census_tract_138
...,...,...,...,...,...
63,Rocky's Wing Shack,restaurant,36.178329,-86.798754,census_tract_138
64,Alexandria Grill & Pizza,restaurant,36.180131,-86.798522,census_tract_138
65,A Notable Event,night_club,36.180915,-86.822564,census_tract_138
66,G-SPOT,night_club,36.179914,-86.813541,census_tract_138


In [99]:
location_139= '36.17571, -86.80562'
radius_139= '1514'

In [100]:
ct139 = []
ct139_list = []
latitude139 = []
longitude139 = []
name139 = []
type139 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct139_res = search_places_by_coordinate(location_139, radius_139, i, api_key) # running the API function through each type
    ct139.append(ct139_res) # creating a lists of the lists of each type of business
    for list in ct139:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude139.append(element['geometry']['location']['lat'])
            longitude139.append(element['geometry']['location']['lng'])
            name139.append(element['name'])
            type139.append(element['types'][0])

In [101]:
df139 = pd.DataFrame(zip(name139, type139, latitude139, longitude139),
               columns =['name', 'type', 'latitude', 'longitude'])

In [102]:
df139.drop_duplicates(inplace=True, ignore_index=True)
df139

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.175368,-86.793211
1,bp,gas_station,36.172959,-86.791613
2,Pauls Market,gas_station,36.169698,-86.807180
3,Kings Market,gas_station,36.169579,-86.807004
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625
...,...,...,...,...
84,G-SPOT,night_club,36.179914,-86.813541
85,EG&MC,bar,36.172862,-86.793766
86,bar,bar,36.175811,-86.790228
87,swerf333,bar,36.175811,-86.790228


In [103]:
df139['tract'] = 'census_tract_139'
df139

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_139
1,bp,gas_station,36.172959,-86.791613,census_tract_139
2,Pauls Market,gas_station,36.169698,-86.807180,census_tract_139
3,Kings Market,gas_station,36.169579,-86.807004,census_tract_139
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_139
...,...,...,...,...,...
84,G-SPOT,night_club,36.179914,-86.813541,census_tract_139
85,EG&MC,bar,36.172862,-86.793766,census_tract_139
86,bar,bar,36.175811,-86.790228,census_tract_139
87,swerf333,bar,36.175811,-86.790228,census_tract_139


In [104]:
location_142= '36.16693, -86.80365'
radius_142= '1640'

In [105]:
ct142 = []
ct142_list = []
latitude142 = []
longitude142 = []
name142 = []
type142 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct142_res = search_places_by_coordinate(location_142, radius_142, i, api_key) # running the API function through each type
    ct142.append(ct142_res) # creating a lists of the lists of each type of business
    for list in ct142:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude142.append(element['geometry']['location']['lat'])
            longitude142.append(element['geometry']['location']['lng'])
            name142.append(element['name'])
            type142.append(element['types'][0])

In [106]:
df142 = pd.DataFrame(zip(name142, type142, latitude142, longitude142),
               columns =['name', 'type', 'latitude', 'longitude'])

In [107]:
df142.drop_duplicates(inplace=True, ignore_index=True)
df142

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,bp,gas_station,36.172959,-86.791613
2,Kroger Fuel Center,gas_station,36.175368,-86.793211
3,Exxon,gas_station,36.160241,-86.793982
4,Pauls Market,gas_station,36.169698,-86.807180
...,...,...,...,...
175,Minerva Avenue,bar,36.180650,-86.803357
176,Willie B's Kitchen & Lounge,restaurant,36.180752,-86.802850
177,bar,bar,36.175811,-86.790228
178,swerf333,bar,36.175811,-86.790228


In [108]:
df142['tract'] = 'census_tract_142'
df142

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_142
1,bp,gas_station,36.172959,-86.791613,census_tract_142
2,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_142
3,Exxon,gas_station,36.160241,-86.793982,census_tract_142
4,Pauls Market,gas_station,36.169698,-86.807180,census_tract_142
...,...,...,...,...,...
175,Minerva Avenue,bar,36.180650,-86.803357,census_tract_142
176,Willie B's Kitchen & Lounge,restaurant,36.180752,-86.802850,census_tract_142
177,bar,bar,36.175811,-86.790228,census_tract_142
178,swerf333,bar,36.175811,-86.790228,census_tract_142


In [109]:
location_143= '36.16124, -86.81564'
radius_143= '1552'

In [110]:
ct143 = []
ct143_list = []
latitude143 = []
longitude143 = []
name143 = []
type143 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct143_res = search_places_by_coordinate(location_143, radius_143, i, api_key) # running the API function through each type
    ct143.append(ct143_res) # creating a lists of the lists of each type of business
    for list in ct143:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude143.append(element['geometry']['location']['lat'])
            longitude143.append(element['geometry']['location']['lng'])
            name143.append(element['name'])
            type143.append(element['types'][0])

In [111]:
df143 = pd.DataFrame(zip(name143, type143, latitude143, longitude143),
               columns =['name', 'type', 'latitude', 'longitude'])

In [112]:
df143.drop_duplicates(inplace=True, ignore_index=True)
df143

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,Pauls Market,gas_station,36.169698,-86.807180
2,Marathon Gas,gas_station,36.167704,-86.820956
3,Marathon Gas,gas_station,36.167786,-86.821652
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625
...,...,...,...,...
139,Thighstop,restaurant,36.151585,-86.803262
140,Wingstop,restaurant,36.151585,-86.803262
141,SandBar Nashville,bar,36.152374,-86.820015
142,The End,bar,36.151108,-86.803811


In [113]:
df143['tract'] = 'census_tract_143'
df143

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_143
1,Pauls Market,gas_station,36.169698,-86.807180,census_tract_143
2,Marathon Gas,gas_station,36.167704,-86.820956,census_tract_143
3,Marathon Gas,gas_station,36.167786,-86.821652,census_tract_143
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_143
...,...,...,...,...,...
139,Thighstop,restaurant,36.151585,-86.803262,census_tract_143
140,Wingstop,restaurant,36.151585,-86.803262,census_tract_143
141,SandBar Nashville,bar,36.152374,-86.820015,census_tract_143
142,The End,bar,36.151108,-86.803811,census_tract_143


In [114]:
location_144= '36.15766, -86.80573'
radius_144= '2194'

In [115]:
ct144 = []
ct144_list = []
latitude144 = []
longitude144 = []
name144 = []
type144 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct144_res = search_places_by_coordinate(location_144, radius_144, i, api_key) # running the API function through each type
    ct144.append(ct144_res) # creating a lists of the lists of each type of business
    for list in ct144:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude144.append(element['geometry']['location']['lat'])
            longitude144.append(element['geometry']['location']['lng'])
            name144.append(element['name'])
            type144.append(element['types'][0])

In [116]:
df144 = pd.DataFrame(zip(name144, type144, latitude144, longitude144),
               columns =['name', 'type', 'latitude', 'longitude'])

In [117]:
df144.drop_duplicates(inplace=True, ignore_index=True)
df144

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,Shell,gas_station,36.153856,-86.794865
2,Exxon,gas_station,36.160241,-86.793982
3,bp,gas_station,36.172959,-86.791613
4,Exxon,gas_station,36.156550,-86.786897
...,...,...,...,...
238,Commodore Grille,bar,36.144992,-86.810991
239,Cinco De Mayo Mexican Restaurant Downtown,bar,36.154544,-86.782032
240,Library Bar,bar,36.162321,-86.781994
241,Bobby's Idle Hour Tavern,bar,36.146530,-86.793333


In [118]:
df144['tract'] = 'census_tract_144'
df144

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_144
1,Shell,gas_station,36.153856,-86.794865,census_tract_144
2,Exxon,gas_station,36.160241,-86.793982,census_tract_144
3,bp,gas_station,36.172959,-86.791613,census_tract_144
4,Exxon,gas_station,36.156550,-86.786897,census_tract_144
...,...,...,...,...,...
238,Commodore Grille,bar,36.144992,-86.810991,census_tract_144
239,Cinco De Mayo Mexican Restaurant Downtown,bar,36.154544,-86.782032,census_tract_144
240,Library Bar,bar,36.162321,-86.781994,census_tract_144
241,Bobby's Idle Hour Tavern,bar,36.146530,-86.793333,census_tract_144


In [119]:
location_148= '36.15109, -86.75970'
radius_148= '1430'

In [120]:
ct148 = []
ct148_list = []
latitude148 = []
longitude148 = []
name148 = []
type148 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct148_res = search_places_by_coordinate(location_148, radius_148, i, api_key) # running the API function through each type
    ct148.append(ct148_res) # creating a lists of the lists of each type of business
    for list in ct148:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude148.append(element['geometry']['location']['lat'])
            longitude148.append(element['geometry']['location']['lng'])
            name148.append(element['name'])
            type148.append(element['types'][0])

In [121]:
df148 = pd.DataFrame(zip(name148, type148, latitude148, longitude148),
               columns =['name', 'type', 'latitude', 'longitude'])

In [122]:
df148.drop_duplicates(inplace=True, ignore_index=True)
df148

,name,type,latitude,longitude
0,Shell Island,gas_station,36.147766,-86.758112
1,GC MART,gas_station,36.147725,-86.758041
2,Ebt,gas_station,36.147338,-86.757676
3,Mucho Express,convenience_store,36.146841,-86.757527
4,Eagle Parking - Shell,gas_station,36.157081,-86.770247
...,...,...,...,...
87,Lucky's 3 Star Bar,bar,36.139924,-86.763536
88,Porch Coffee & Market,cafe,36.157509,-86.772627
89,Cocktail & Key,bar,36.141827,-86.769662
90,Sidebar,bar,36.157453,-86.772731


In [123]:
df148['tract'] = 'census_tract_148'
df148

,name,type,latitude,longitude,tract
0,Shell Island,gas_station,36.147766,-86.758112,census_tract_148
1,GC MART,gas_station,36.147725,-86.758041,census_tract_148
2,Ebt,gas_station,36.147338,-86.757676,census_tract_148
3,Mucho Express,convenience_store,36.146841,-86.757527,census_tract_148
4,Eagle Parking - Shell,gas_station,36.157081,-86.770247,census_tract_148
...,...,...,...,...,...
87,Lucky's 3 Star Bar,bar,36.139924,-86.763536,census_tract_148
88,Porch Coffee & Market,cafe,36.157509,-86.772627,census_tract_148
89,Cocktail & Key,bar,36.141827,-86.769662,census_tract_148
90,Sidebar,bar,36.157453,-86.772731,census_tract_148


In [124]:
location_15630= '36.03003, -86.61451'
radius_15630= '3149'

In [125]:
ct15630 = []
ct15630_list = []
latitude15630 = []
longitude15630 = []
name15630 = []
type15630 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct15630_res = search_places_by_coordinate(location_15630, radius_15630, i, api_key) # running the API function through each type
    ct15630.append(ct15630_res) # creating a lists of the lists of each type of business
    for list in ct15630:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude15630.append(element['geometry']['location']['lat'])
            longitude15630.append(element['geometry']['location']['lng'])
            name15630.append(element['name'])
            type15630.append(element['types'][0])

In [126]:
df15630 = pd.DataFrame(zip(name15630, type15630, latitude15630, longitude15630),
               columns =['name', 'type', 'latitude', 'longitude'])

In [127]:
df15630.drop_duplicates(inplace=True, ignore_index=True)
df15630

,name,type,latitude,longitude
0,Shell,gas_station,36.016064,-86.620556
1,bp,gas_station,36.015064,-86.618748
2,Family Mart,gas_station,36.022759,-86.584634
3,TA Travel Center,gas_station,36.014914,-86.618405
4,Exxon,gas_station,36.029188,-86.590429
5,Quick Stop,grocery_or_supermarket,36.042953,-86.607675
6,bp,gas_station,36.042936,-86.607383
7,Exxon,gas_station,36.016439,-86.619420
8,Thorntons,convenience_store,36.016627,-86.620823
9,Exxon,gas_station,36.029162,-86.591431


In [128]:
df15630['tract'] = 'census_tract_15630'
df15630

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.016064,-86.620556,census_tract_15630
1,bp,gas_station,36.015064,-86.618748,census_tract_15630
2,Family Mart,gas_station,36.022759,-86.584634,census_tract_15630
3,TA Travel Center,gas_station,36.014914,-86.618405,census_tract_15630
4,Exxon,gas_station,36.029188,-86.590429,census_tract_15630
5,Quick Stop,grocery_or_supermarket,36.042953,-86.607675,census_tract_15630
6,bp,gas_station,36.042936,-86.607383,census_tract_15630
7,Exxon,gas_station,36.016439,-86.619420,census_tract_15630
8,Thorntons,convenience_store,36.016627,-86.620823,census_tract_15630
9,Exxon,gas_station,36.029162,-86.591431,census_tract_15630


In [129]:
location_162= '36.13994, -86.78199'
radius_162= '1854'

In [130]:
ct162 = []
ct162_list = []
latitude162 = []
longitude162 = []
name162 = []
type162 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct162_res = search_places_by_coordinate(location_162, radius_162, i, api_key) # running the API function through each type
    ct162.append(ct162_res) # creating a lists of the lists of each type of business
    for list in ct162:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude162.append(element['geometry']['location']['lat'])
            longitude162.append(element['geometry']['location']['lng'])
            name162.append(element['name'])
            type162.append(element['types'][0])

In [131]:
df162 = pd.DataFrame(zip(name162, type162, latitude162, longitude162),
               columns =['name', 'type', 'latitude', 'longitude'])

In [132]:
df162.drop_duplicates(inplace=True, ignore_index=True)
df162

,name,type,latitude,longitude
0,MAPCO Mart,gas_station,36.134648,-86.779522
1,Shell,gas_station,36.132442,-86.779478
2,Exxon,gas_station,36.132638,-86.795786
3,bp,gas_station,36.134567,-86.779460
4,Circle K,convenience_store,36.132559,-86.795966
...,...,...,...,...
220,Barcelona Wine Bar,bar,36.142875,-86.791739
221,Jack Brown's Beer & Burger Joint Edgehill,bar,36.143006,-86.792333
222,Fat Kat Slim's,restaurant,36.149475,-86.778811
223,Diskin Cider,bar,36.140849,-86.769778


In [133]:
df162['tract'] = 'census_tract_162'
df162

,name,type,latitude,longitude,tract
0,MAPCO Mart,gas_station,36.134648,-86.779522,census_tract_162
1,Shell,gas_station,36.132442,-86.779478,census_tract_162
2,Exxon,gas_station,36.132638,-86.795786,census_tract_162
3,bp,gas_station,36.134567,-86.779460,census_tract_162
4,Circle K,convenience_store,36.132559,-86.795966,census_tract_162
...,...,...,...,...,...
220,Barcelona Wine Bar,bar,36.142875,-86.791739,census_tract_162
221,Jack Brown's Beer & Burger Joint Edgehill,bar,36.143006,-86.792333,census_tract_162
222,Fat Kat Slim's,restaurant,36.149475,-86.778811,census_tract_162
223,Diskin Cider,bar,36.140849,-86.769778,census_tract_162


In [134]:
location_193= '36.17114, -86.76886'
radius_193= '2977'

In [135]:
ct193 = []
ct193_list = []
latitude193 = []
longitude193 = []
name193 = []
type193 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct193_res = search_places_by_coordinate(location_193, radius_193, i, api_key) # running the API function through each type
    ct193.append(ct193_res) # creating a lists of the lists of each type of business
    for list in ct193:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude193.append(element['geometry']['location']['lat'])
            longitude193.append(element['geometry']['location']['lng'])
            name193.append(element['name'])
            type193.append(element['types'][0])

In [136]:
df193 = pd.DataFrame(zip(name193, type193, latitude193, longitude193),
               columns =['name', 'type', 'latitude', 'longitude'])

In [137]:
df193.drop_duplicates(inplace=True, ignore_index=True)
df193

,name,type,latitude,longitude
0,TA Travel Center,gas_station,36.171075,-86.771986
1,Trillium,gas_station,36.178444,-86.774921
2,Shell,gas_station,36.171552,-86.751873
3,Express Food Mart,convenience_store,36.177783,-86.749592
4,Marathon,gas_station,36.177779,-86.749787
...,...,...,...,...
180,AJ's Good Time Bar,bar,36.160511,-86.777863
181,Mellow Mushroom,meal_takeaway,36.160409,-86.777885
182,The Oak Bar,bar,36.163679,-86.782314
183,Von Elrod's Beer Hall & Kitchen,bar,36.173755,-86.785026


In [138]:
df193['tract'] = 'census_tract_193'
df193

,name,type,latitude,longitude,tract
0,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_193
1,Trillium,gas_station,36.178444,-86.774921,census_tract_193
2,Shell,gas_station,36.171552,-86.751873,census_tract_193
3,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_193
4,Marathon,gas_station,36.177779,-86.749787,census_tract_193
...,...,...,...,...,...
180,AJ's Good Time Bar,bar,36.160511,-86.777863,census_tract_193
181,Mellow Mushroom,meal_takeaway,36.160409,-86.777885,census_tract_193
182,The Oak Bar,bar,36.163679,-86.782314,census_tract_193
183,Von Elrod's Beer Hall & Kitchen,bar,36.173755,-86.785026,census_tract_193


## Now I will make a list of all the dataframes and concatenate them into one.

In [139]:
all_tracts = [df10105,
              df10106,
              df10903,
              df10904,
              df11001,
              df114,
              df118,
              df119,
              df126,
              df12701,
              df12702,
              df12801,
              df12802,
              df13601,
              df13602,
              df137,
              df138,
              df139,
              df142,
              df143,
              df144,
              df148,
              df15630,
              df162,
              df193]

In [140]:
businesses_pred_black_tracts = pd.concat(all_tracts)

In [141]:
businesses_pred_black_tracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2605 entries, 0 to 184
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       2605 non-null   object 
 1   type       2605 non-null   object 
 2   latitude   2605 non-null   float64
 3   longitude  2605 non-null   float64
 4   tract      2605 non-null   object 
dtypes: float64(2), object(3)
memory usage: 122.1+ KB


## Since I ran the API multiple times and used a buffer, I may have some duplicate businesses across tracts. I want to drop duplicates from the entire dataframe. 

In [142]:
unique_businesses  = businesses_pred_black_tracts.drop_duplicates(
  subset = ['latitude', 'longitude', 'name'],
  keep = 'first').reset_index(drop = True)

In [143]:
unique_businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       1405 non-null   object 
 1   type       1405 non-null   object 
 2   latitude   1405 non-null   float64
 3   longitude  1405 non-null   float64
 4   tract      1405 non-null   object 
dtypes: float64(2), object(3)
memory usage: 55.0+ KB


## I will save this as a csv. In another notebook I will turn it into a GeoPandasDataFrame and merge it with the race GPD so we can use it for some map visualizations.

In [144]:
unique_businesses.to_csv('../data/businesses_in_pred_black_tracts_2.csv')